# Capstone Project
#### Identifying districts in London that are similar to a chosen New York neighborhood
_By Elena Tuchina_

***

### Table of Contents

[Import Libraries](#Import-Libraries)
    
[Download New York Dataset](#Download-New-York-Dataset)

[Download London Dataset](#Download-London-Dataset)
    
[Get Foursquare Data](#Get-Foursquare-Data)

[Cluster Analysis](#Cluster-Analysis)


***

### Import Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!pip install Beautifulsoup4
#!pip install lxml # HTML parser
from bs4 import BeautifulSoup # library to scrape HTML pages

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!pip install geocoder
#import geocoder # import geocoder

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

from sklearn.cluster import KMeans # import k-means from clustering stage

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


***

### Download New York Dataset

New York neighborhood dataset is taken from NYU Spatial Data Repository (https://geo.nyu.edu/catalog/nyu_2451_34572) and contains the boroughs and the neighborhoods that exist in each borough as well as the the latitude and longitude coordinates of each neighborhood.

In [3]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_neighborhoods = json.load(json_data)
newyork_neighborhoods = newyork_neighborhoods['features']

In [5]:
newyork_neighborhoods[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# instantiate the dataframe
column_names = ['Area', 'Borough', 'Latitude', 'Longitude'] 
newyork_data = pd.DataFrame(columns=column_names)

for data in newyork_neighborhoods:
    area = data['properties']['name']
    borough = data['properties']['borough'] 
    print (borough)
    area_latlon = data['geometry']['coordinates']
    area_lat = area_latlon[1]
    area_lon = area_latlon[0]

    newyork_data = newyork_data.append({'Area': area,
                                        'Borough': borough,
                                        'Latitude': area_lat,
                                        'Longitude': area_lon}, ignore_index=True)
newyork_data.head()

Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Manhattan
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Bronx
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Brooklyn
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manhattan
Manh

,Area,Borough,Latitude,Longitude
0,Wakefield,Bronx,40.894705,-73.847201
1,Co-op City,Bronx,40.874294,-73.829939
2,Eastchester,Bronx,40.887556,-73.827806
3,Fieldston,Bronx,40.895437,-73.905643
4,Riverdale,Bronx,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} areas.'.format(
        len(newyork_data['Borough'].unique()),
        newyork_data.shape[0]
    )
)

The dataframe has 5 boroughs and 306 areas.


***

### Download London Dataset

The London district dataset is taken from Wikipedia (https://en.wikipedia.org/wiki/List_of_areas_of_London) and contains the boroughs and the districts that exist in each borough. Only Inner London boroughs are used in this analysis. Nominatum geolocator from Geopy is used to find latitude and longitude coordinates of each districts.

In [10]:
#parse HTML page
webpage = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
source = requests.get(webpage).text
soup = BeautifulSoup(source, 'lxml')

In [11]:
# converting HTLM table into a dataframe

table = soup.find('table', class_='wikitable')

rows = table.find_all('tr')
del rows[0]

# instantiate the dataframe
col_names=['Area','Borough']
london_data = pd.DataFrame(columns=col_names)

for row in rows:
    cols = row.find_all('td')
    area = cols[0].text.partition('(')[0]
    borough = cols[1].text.partition(',')[0].partition('[')[0]
    
    london_data = london_data.append({'Area': area,
                        'Borough': borough}, ignore_index=True)

In [12]:
print(london_data.shape)
london_data.head()

(533, 2)


,Area,Borough
0,Abbey Wood,Greenwich
1,Acton,Ealing
2,Addington,Croydon
3,Addiscombe,Croydon
4,Albany Park,Bexley


In [16]:
#leaving only Inner London boroughs
InnerBoroughs = ['City', 'Camden', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Newham', 'Southwark', 'Tower Hamlets','Wandsworth', 'Westminster']
london_data = london_data.loc[london_data['Borough'].isin(InnerBoroughs)].reset_index(drop=True)

# correcting for name misspeling
london_data = london_data.replace('Somerstown', 'Somers Town')

# adding Latutude and Longitude columns to the dataframe
london_data['Latitude'] = 0.0
london_data['Longitude'] = 0.0
london_data.head()

,Area,Borough,Latitude,Longitude
0,Aldgate,City,0.0,0.0
1,Aldwych,Westminster,0.0,0.0
2,Balham,Wandsworth,0.0,0.0
3,Bankside,Southwark,0.0,0.0
4,Barbican,City,0.0,0.0


In [17]:
# Get lon/lat coordinates

geolocator = Nominatim()

for index, row in london_data.iterrows():
    address = row['Area'] + ', ' + row['Borough'] + ', UK'
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    if (latitude < 51.0) or (longitude > 1.0):
        address = row['Area'] + ', UK'
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    london_data.at[index, 'Latitude'] = latitude
    london_data.at[index, 'Longitude'] = longitude
    print('Coordinates for {} are {}, {}'.format(address, latitude, longitude))

london_data.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Coordinates for Aldgate, City, UK are 51.5142477, -0.0757186
Coordinates for Aldwych, Westminster, UK are 51.5129401, -0.1181008
Coordinates for Balham, Wandsworth, UK are 51.4428285, -0.1514426
Coordinates for Bankside, Southwark, UK are 51.508302, -0.0962967
Coordinates for Barbican, City, UK are 51.5201501, -0.0986832
Coordinates for Battersea, Wandsworth, UK are 51.4707933, -0.172214
Coordinates for Bayswater, Westminster, UK are 51.5124135, -0.1876323
Coordinates for Beckton, Newham, UK are 51.5160797, 0.0594257
Coordinates for Belgravia, Westminster, UK are 51.4982128, -0.1534998
Coordinates for Bellingham, Lewisham, UK are 51.4329965, -0.0193376
Coordinates for Belsize Park, Camden, UK are 51.5473926, -0.1683005
Coordinates for Bermondsey, Southwark, UK are 51.4970125, -0.063268
Coordinates for Bethnal Green, Tower Hamlets, UK are 51.5303456, -0.0561633
Coordinates for Blackfriars, City, UK are 51.5115854, -0.1037671
Coordinates for Blackheath, Lewisham, UK are 51.4663175, 0.008

,Area,Borough,Latitude,Longitude
0,Aldgate,City,51.514248,-0.075719
1,Aldwych,Westminster,51.512940,-0.118101
2,Balham,Wandsworth,51.442828,-0.151443
3,Bankside,Southwark,51.508302,-0.096297
4,Barbican,City,51.520150,-0.098683


In [18]:
# create map of London

#geolocator = Nominatim()
address = 'London, UK'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

london_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add disricts as markers to map
for lat, lng, district, borough in zip(london_data['Latitude'], london_data['Longitude'], london_data['Area'], london_data['Borough']):
    label = '{}, {}'.format(district, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london_map)  

london_map

***

### Get Foursquare Data

In [19]:
# set API credentials
CLIENT_ID = 'XPWW0VGBU2F5XPERQWY1RG5LMB4HGQFAPSZZK5DXZ3DASFTZ' # your Foursquare ID
CLIENT_SECRET = 'TES1JPBUX2UHIR1ZZWG3QMHCAOCOGDSFHKRXO5S5POBQVTAR' # your Foursquare Secret
VERSION = '20181101' # Foursquare API version

In [20]:
# define a function for getting information on nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Obtain list of venues by area for London

In [21]:
newyork_venues = getNearbyVenues(names = newyork_data['Area'],
                                latitudes = newyork_data['Latitude'],
                                longitudes = newyork_data['Longitude']
                                )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [40]:
print(newyork_venues.shape)
newyork_venues.head()

(10211, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop


In [25]:
# drop the areas with less than 100 reported venues
newyork_venues = newyork_venues.groupby('Area').filter(lambda x: len(x)>=100)

print(newyork_venues.shape)
newyork_venues.groupby('Area').count()

(9760, 7)


,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
Astoria,100,100,100,100,100,100
Auburndale,100,100,100,100,100,100
Battery Park City,100,100,100,100,100,100
Bay Ridge,100,100,100,100,100,100
Baychester,100,100,100,100,100,100
Bayside,100,100,100,100,100,100
Bedford Stuyvesant,100,100,100,100,100,100
Belmont,100,100,100,100,100,100
Bensonhurst,100,100,100,100,100,100


In [26]:
print('The resulting dataframe has {} areas.'.format(
        newyork_venues.groupby('Area').count().shape[0]
    )
)

The resulting dataframe has 96 areas.


In [27]:
print('There are {} uniques venue categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 390 uniques venue categories.


#### Obtain list of venues by area for London

In [29]:
london_venues = getNearbyVenues(names = london_data['Area'],
                                latitudes = london_data['Latitude'],
                                longitudes = london_data['Longitude']
                                )

Aldgate
Aldwych
Balham
Bankside
Barbican
Battersea
Bayswater
Beckton
Belgravia
Bellingham
Belsize Park
Bermondsey
Bethnal Green
Blackfriars
Blackheath
Blackwall
Bloomsbury
Bounds Green
Bow
Bowes Park
Brixton
Brockley
Bromley 
Brompton
Camberwell
Cambridge Heath
Camden Town
Canary Wharf
Canning Town
Catford
Chalk Farm
Charing Cross
Chelsea
Chinatown
Chinbrook
Clapham
Covent Garden
Crofton Park
Crouch End
Cubitt Town
Custom House
Dalston
Denmark Hill
Deptford
Downham
Dulwich
Earls Court
Earlsfield
East Dulwich
East Ham
Elephant and Castle
Finsbury Park
Fitzrovia
Forest Gate
Forest Hill
Fortis Green
Frognal
Fulham
Gipsy Hill
Gospel Oak
Grove Park
Hackney
Hackney Central
Hackney Marshes
Hackney Wick
Haggerston
Hammersmith
Hampstead
Harringay
Herne Hill
Highgate
Hither Green
Holborn
Holland Park
Homerton
Honor Oak
Hornsey
Hoxton
Isle of Dogs
Kennington
Kensington
Kentish Town
Knightsbridge
Ladywell
Lambeth
Lea Bridge
Leamouth
Lee
Lewisham
Limehouse
Lisson Grove
Little Ilford
Little Venice
L

In [30]:
# drop the areas with less than 100 reported venues
london_venues = london_venues.groupby('Area').filter(lambda x: len(x)>=100)

print(london_venues.shape)
london_venues.groupby('Area').count()

(9000, 7)


,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
Aldgate,100,100,100,100,100,100
Aldwych,100,100,100,100,100,100
Bankside,100,100,100,100,100,100
Barbican,100,100,100,100,100,100
Battersea,100,100,100,100,100,100
Bayswater,100,100,100,100,100,100
Belgravia,100,100,100,100,100,100
Belsize Park,100,100,100,100,100,100
Bermondsey,100,100,100,100,100,100


In [31]:
print('The resulting dataframe has {} areas.'.format(
        london_venues.groupby('Area').count().shape[0]
    )
)

The resulting dataframe has 90 areas.


In [32]:
print('There are {} uniques venue categories.'.format(len(london_venues['Venue Category'].unique())))

There are 340 uniques venue categories.


***

### Cluster Analysis 

##### Select a New York neighborhood for similarity analysis with London districts

In [33]:
print(newyork_venues['Area'].unique())

['Kingsbridge' 'Baychester' 'Fordham' 'Belmont' 'Bay Ridge' 'Bensonhurst'
 'Sunset Park' 'Greenpoint' 'Sheepshead Bay' 'Flatbush' 'Crown Heights'
 'Kensington' 'Prospect Heights' 'Williamsburg' 'Bushwick'
 'Bedford Stuyvesant' 'Brooklyn Heights' 'Cobble Hill' 'Carroll Gardens'
 'Gowanus' 'Fort Greene' 'Park Slope' 'Clinton Hill' 'Downtown'
 'Boerum Hill' 'Prospect Lefferts Gardens' 'Prospect Park South'
 'East Williamsburg' 'North Side' 'South Side' 'Fort Hamilton' 'Chinatown'
 'Washington Heights' 'Inwood' 'Hamilton Heights' 'Manhattanville'
 'Central Harlem' 'East Harlem' 'Upper East Side' 'Yorkville' 'Lenox Hill'
 'Roosevelt Island' 'Upper West Side' 'Lincoln Square' 'Clinton' 'Midtown'
 'Murray Hill' 'Chelsea' 'Greenwich Village' 'East Village'
 'Lower East Side' 'Tribeca' 'Little Italy' 'Soho' 'West Village'
 'Manhattan Valley' 'Morningside Heights' 'Gramercy' 'Battery Park City'
 'Financial District' 'Astoria' 'Woodside' 'Jackson Heights' 'Elmhurst'
 'Corona' 'Forest Hills' 'Flus

In [35]:
neighborhood = input('Enter neighborhood name from the list above: ')
print('You have chosen to perform analysis for: ', neighborhood)

Enter neighborhood name from the list above:  Upper West Side


You have chosen to perform analysis for:  Upper West Side


In [36]:
newyork_data.loc[newyork_data['Area'] == neighborhood]

,Area,Borough,Latitude,Longitude
111,Upper West Side,Manhattan,40.787658,-73.977059


In [37]:
neighborhood_venues = newyork_venues[newyork_venues['Area'] == neighborhood]
print (neighborhood_venues.shape)
neighborhood_venues.head()

(100, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
8502,Upper West Side,40.787658,-73.977059,Jacob's Pickles,40.786626,-73.975523,Southern / Soul Food Restaurant
8503,Upper West Side,40.787658,-73.977059,AMC Loews 84th Street 6,40.786795,-73.977640,Movie Theater
8504,Upper West Side,40.787658,-73.977059,Maison Pickle,40.786990,-73.977787,American Restaurant
8505,Upper West Side,40.787658,-73.977059,Han Dynasty,40.787620,-73.976359,Chinese Restaurant
8506,Upper West Side,40.787658,-73.977059,Barney Greengrass,40.788008,-73.974794,Bagel Shop


In [38]:
print(london_venues.shape)
london_venues.head()

(9000, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldgate,51.514248,-0.075719,The Association,51.513733,-0.079132,Coffee Shop
1,Aldgate,51.514248,-0.075719,Benk + Bo,51.515731,-0.075875,Bakery
2,Aldgate,51.514248,-0.075719,Discount Suit Company,51.516705,-0.075506,Cocktail Bar
3,Aldgate,51.514248,-0.075719,Hotel Indigo,51.512782,-0.075871,Hotel
4,Aldgate,51.514248,-0.075719,"Dorsett City, London",51.514008,-0.075715,Hotel


In [39]:
# add New York neighborhood venues to London venues dataframe
venues = pd.concat([london_venues, neighborhood_venues]).reset_index(drop=True)
print(venues.shape)
venues.head()

(9100, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldgate,51.514248,-0.075719,The Association,51.513733,-0.079132,Coffee Shop
1,Aldgate,51.514248,-0.075719,Benk + Bo,51.515731,-0.075875,Bakery
2,Aldgate,51.514248,-0.075719,Discount Suit Company,51.516705,-0.075506,Cocktail Bar
3,Aldgate,51.514248,-0.075719,Hotel Indigo,51.512782,-0.075871,Hotel
4,Aldgate,51.514248,-0.075719,"Dorsett City, London",51.514008,-0.075715,Hotel


In [40]:
print('There are {} uniques venue categories.'.format(len(venues['Venue Category'].unique())))

There are 345 uniques venue categories.


In [41]:
# one hot encoding
london_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Area'] = venues['Area']

col_num = london_onehot.columns.get_loc('Area')

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[col_num]]+ list(london_onehot.columns[0:col_num]) 
fixed_columns = fixed_columns + list(london_onehot.columns[col_num+1:-1]) 

london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Area,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaz

In [42]:
london_onehot.shape

(9100, 346)

In [43]:
london_grouped = london_onehot.groupby('Area').mean().reset_index()
london_grouped

,Area,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Colombian Restaurant,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaz

In [44]:
london_grouped.shape

(91, 346)

In [45]:
num_top_venues = 5

for area in london_grouped['Area']:
    print("----"+area+"----")
    temp = london_grouped[london_grouped['Area'] == area].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aldgate----
               venue  freq
0              Hotel  0.11
1        Coffee Shop  0.09
2         Restaurant  0.05
3        Salad Place  0.04
4  Indian Restaurant  0.04


----Aldwych----
          venue  freq
0       Theater  0.09
1   Coffee Shop  0.06
2         Hotel  0.06
3  Dessert Shop  0.04
4           Pub  0.03


----Bankside----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.05
2  Seafood Restaurant  0.05
3                 Pub  0.04
4               Hotel  0.04


----Barbican----
                   venue  freq
0            Coffee Shop  0.11
1                    Pub  0.07
2   Gym / Fitness Center  0.07
3  Vietnamese Restaurant  0.05
4                  Hotel  0.04


----Battersea----
                  venue  freq
0                   Pub  0.11
1         Grocery Store  0.07
2                 Hotel  0.06
3           Coffee Shop  0.05
4  Gym / Fitness Center  0.04


----Bayswater----
                venue  freq
0                 Pub  0.09
1   

In [48]:
# define the function for determining 10 most frequently returned venue categories 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
areas_venues_sorted = pd.DataFrame(columns=columns)
areas_venues_sorted['Area'] = london_grouped['Area']

for ind in np.arange(london_grouped.shape[0]):
    areas_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

areas_venues_sorted

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldgate,Hotel,Coffee Shop,Restaurant,Pub,Salad Place,Indian Restaurant,Gym / Fitness Center,Cupcake Shop,Cocktail Bar,Hotel Bar
1,Aldwych,Theater,Hotel,Coffee Shop,Dessert Shop,Restaurant,History Museum,Pub,French Restaurant,Clothing Store,Cocktail Bar
2,Bankside,Coffee Shop,Italian Restaurant,Seafood Restaurant,Hotel,Pub,Gym / Fitness Center,Food Truck,Portuguese Restaurant,Bakery,Art Museum
3,Barbican,Coffee Shop,Pub,Gym / Fitness Center,Vietnamese Restaurant,Hotel,Café,Food Truck,Italian Restaurant,Steakhouse,Scenic Lookout
4,Battersea,Pub,Grocery Store,Hotel,Coffee Shop,Café,Gym / Fitness Center,Pizza Place,Bar,Bakery,Supermarket
5,Bayswater,Pub,Hotel,Greek Restaurant,Garden,Coffee Shop,Chinese Restaurant,Burger Joint,Gym / Fitness Center,Restaurant,Café
6,Belgravia,Hotel,Restaurant,Italian Restaurant,Café,Boutique,Plaza,Lounge,Bakery,French Restaurant,Gastropub
7,Belsize Park,Café,Italian Restaurant,Coffee Shop,Bakery,Pub,Grocery Store,Gym / Fitness Center,Pizza Place,Bookstore,Dessert Shop
8,Bermondsey,Pub,Brewery,Coffee Shop,Grocery Store,Park,Bar,Bus Stop,Café,Bakery,Cocktail Bar
9,Bethnal Green,Coffee Shop,Pub,Café,Cocktail Bar,Restaurant,Wine Bar,Bookstore,Pizza Place,Flower Shop,Convenience Store


In [80]:
# set number of clusters
kclusters = 9

london_grouped_clustering = london_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 2, 2, 6, 7, 1, 0, 5, 3], dtype=int32)

In [81]:
areas_venues_sorted.loc[areas_venues_sorted['Area'] == neighborhood]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,Upper West Side,Italian Restaurant,Ice Cream Shop,Coffee Shop,American Restaurant,Wine Bar,Bakery,Bar,Park,Gym,Thai Restaurant


In [82]:
london_merged = pd.merge(london_data, areas_venues_sorted, on='Area', how='right')

# add clustering labels
london_merged['Cluster Labels'] = kmeans.labels_

london_merged.shape

(91, 15)

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# exclude New York neighborhood from displying on the map
clusters = london_merged[0:-1]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clusters['Latitude'], clusters['Longitude'], clusters['Area'], clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Matching Cluster

In [84]:
cluster = london_merged['Cluster Labels'].loc[london_merged['Area'] == neighborhood]
print('Cluster information for {}: {}.'.format(neighborhood,cluster))

Cluster information for Upper West Side: 90    5
Name: Cluster Labels, dtype: int32.


In [85]:
print(' There are {} London districts that are similar to {} in New York.'.format(london_merged.loc[london_merged['Cluster Labels'] == 5].shape[0],neighborhood))

 There are 9 London districts that are similar to Upper West Side in New York.


In [86]:
london_merged.loc[london_merged['Cluster Labels'] == 4,
                  london_merged.columns[[0] + list(range(4, london_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
1,Aldwych,Theater,Hotel,Coffee Shop,Dessert Shop,Restaurant,History Museum,Pub,French Restaurant,Clothing Store,Cocktail Bar,4
19,Charing Cross,Theater,Hotel,Ice Cream Shop,Plaza,Wine Bar,Lounge,Dessert Shop,Bookstore,Steakhouse,Bakery,4
21,Chinatown,Theater,Hotel,Ice Cream Shop,Steakhouse,Plaza,Bakery,Lounge,Bookstore,Wine Bar,Seafood Restaurant,4
23,Covent Garden,Theater,Hotel,Ice Cream Shop,Wine Bar,Coffee Shop,Steakhouse,French Restaurant,Pub,Bookstore,Beer Bar,4
37,Holborn,Coffee Shop,Theater,Hotel,Pub,Dessert Shop,Indian Restaurant,History Museum,Restaurant,Plaza,Cupcake Shop,4
46,Lambeth,Hotel,Theater,Pub,Coffee Shop,Event Space,Bookstore,Beer Bar,Plaza,Sandwich Place,Korean Restaurant,4
68,Soho,Theater,Ice Cream Shop,Hotel,Bakery,Steakhouse,Pizza Place,Plaza,Lounge,Coffee Shop,Bookstore,4
73,St James's,Hotel,Boutique,Clothing Store,Indian Restaurant,Theater,Dessert Shop,Steakhouse,Plaza,Hotel Bar,French Restaurant,4
74,St Giles,Theater,Coffee Shop,Hotel,Ice Cream Shop,Bookstore,French Restaurant,Dessert Shop,History Museum,Steakhouse,Cocktail Bar,4
80,Temple,Theater,Hotel,Coffee Shop,Pub,Dessert Shop,Restaurant,Cocktail Bar,History Museum,Building,Park,4


***